### Importing required libraries

In [ ]:
!pip install wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import wandb
train_data_directory = '/content/drive/MyDrive/Deeplearning/inaturalist_12K/train/'
test_data_directory = '/content/drive/MyDrive/Deeplearning/inaturalist_12K/val/'

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets.utils import download_url
from torchvision import datasets
from torch.utils.data import DataLoader, ConcatDataset, random_split

#Login wandb

In [ ]:
wandb.login(key='5157ae11e5d243722bc57912a56718dc8ef2f734') #Wandb login with my key

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#Loader

In [ ]:
def train_load_(train_d , train__batch_size ,test__batch_size , val_Dataset , val__batch_size,boolT=True ):

    train_batch_load = DataLoader(
        train_d,
         batch_size = train__batch_size,
        shuffle=boolT)

    batch_size = test__batch_size
    test_batch_load = DataLoader(
        test_data_directory,
        batch_size,
        shuffle=boolT)

    val_batch_load = DataLoader(
        val_Dataset,
        batch_size = val__batch_size,
        shuffle=boolT)
    return train_batch_load, val_batch_load, test_batch_load

#Forward

In [ ]:
def fn(convolutional_layer_1 ,activation_function_1 , maxpooling_layer_1 , convolutional_layer_2 ,activation_function_2 , maxpooling_layer_2 ,convolutional_layer_3 , activation_function_3 ,  maxpooling_layer_3 , convolutional_layer_4 , activation_function_4 ,maxpooling_layer_4 , convolutional_layer_5 ,activation_function_5 , maxpooling_layer_5 , fully_connected_layer_1,dropout_layer_first , fully_connected_layer_2,dropout_layer_second , _softmax, data_x):
    #layer 1
    data_x = convolutional_layer_1(data_x)
    shape_1=data_x.shape[1]
    data_x = activation_function_1(data_x)
    data_x = maxpooling_layer_1(data_x)

    #layer 2
    data_x = convolutional_layer_2(data_x)
    shape_1=data_x.shape[1]
    data_x = activation_function_2(data_x)
    data_x = maxpooling_layer_2(data_x)

    #layer 3
    data_x = convolutional_layer_3(data_x)
    shape_1=data_x.shape[1]
    data_x = activation_function_3(data_x)
    data_x = maxpooling_layer_3(data_x)

    #layer 4
    data_x = convolutional_layer_4(data_x)
    shape_1=data_x.shape[1]
    data_x = activation_function_4(data_x)
    data_x = maxpooling_layer_4(data_x)

    #layer 5
    data_x = convolutional_layer_5(data_x)
    data_x = activation_function_5(data_x)
    shape_0=data_x.shape[1]
    data_x = maxpooling_layer_5(data_x)

    shape_1=data_x.shape[1]
    shape_2=data_x.shape[2]
    shape_3=data_x.shape[3]

    shape_all=shape_1*shape_2*shape_3
    data_x = data_x.view(-1,shape_all)
    data_x = fully_connected_layer_1(data_x)
    data_x = dropout_layer_first(data_x)
    data_x = fully_connected_layer_2(data_x)
    data_x = dropout_layer_second(data_x)
    output = _softmax(data_x)

    return data_x


#Transform Data


In [ ]:
def transform_datasets_(train_data_directory ,train_batch_size ,test_batch_size ,val_batch_size ,val_Dataset  , train_d , image_size, mean , std , degree ):
    transformer = transforms.Compose([
    transforms.Resize(image_size),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.02),
    transforms.RandomRotation(degree),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)

    ])

    augmented_dataset = torchvision.datasets.ImageFolder(root=train_data_directory, transform=transformer)
    aug_d= len(augmented_dataset)
    aug_d= 0.2 *aug_d
    augmented_dataset_size = int(aug_d)
    augmented_dataset, _  =  random_split(augmented_dataset, [augmented_dataset_size, len(augmented_dataset) - augmented_dataset_size])
    train__batch_size=train_batch_size
    train_d = ConcatDataset([train_d, augmented_dataset])
    test__batch_size=test_batch_size
    val__batch_size=val_batch_size
    return train_load_(train_d , train__batch_size ,test__batch_size ,val_Dataset ,val__batch_size, boolT=True)

## Loading the iNatuaralist Dataset

In [ ]:
def data_fetch(apply_data_augmentation, train_data_directory, test_data_directory, train_batch_size, val_batch_size, test_batch_size ,image_size , mean, std):

    trans_model_1 = transforms.Compose([
        transforms.Resize(image_size),  #by default: (256,256)
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    train_d = torchvision.datasets.ImageFolder(root=train_data_directory, transform=trans_model_1)
    temp =len(train_d)
    temp = 0.8* temp
    degree=45
    train_d, val_Dataset = random_split(train_d, [int(temp), len(train_d) - int(temp)])
    if apply_data_augmentation== True:
      return transform_datasets_(train_data_directory ,train_batch_size ,test_batch_size ,val_batch_size , val_Dataset , train_d , image_size, mean , std,degree)
    return train_load_(train_d , train_batch_size ,test_batch_size ,val_Dataset ,val_batch_size, boolT=True)

## Setting device to cuda if available


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Create a CNN Model

In [ ]:
class initialize_lenet_CNN(torch.nn.Module):
    def __init__(self, number_of_classes, size_of_kernel, num_of_filters, activation_function, filter_multiplier,dropout_factor):
        super(initialize_lenet_CNN, self).__init__()

        activation_map = {
        'ReLU': torch.nn.ReLU(),
        'SiLU': torch.nn.SiLU(),
        'Mish': torch.nn.Mish(),
        'GELU': torch.nn.GELU()
         }

        #layer 1
        self.convolutional_layer_1 = torch.nn.Conv2d(in_channels=3, out_channels=num_of_filters, kernel_size=size_of_kernel)
        prev_number_filters = num_of_filters
        self.size = (256-size_of_kernel)
        self.activation_function_1 = activation_map.get(activation_function, torch.nn.ReLU())
        self.maxpooling_layer_1 = torch.nn.MaxPool2d(kernel_size=size_of_kernel, stride=2)
        self.size = self.size//2
        number_filters = int(num_of_filters * filter_multiplier)

        #layer 2
        self.convolutional_layer_2 = torch.nn.Conv2d(in_channels=prev_number_filters, out_channels=number_filters, kernel_size=size_of_kernel)
        self.size = (self.size - size_of_kernel)
        prev_number_filters = number_filters
        number_filters = int(number_filters * filter_multiplier)
        self.activation_function_2 = activation_map.get(activation_function, torch.nn.ReLU())
        self.maxpooling_layer_2 = torch.nn.MaxPool2d(kernel_size=size_of_kernel, stride=2)
        self.size = self.size//2

        #layer 3
        self.convolutional_layer_3 = torch.nn.Conv2d(in_channels=prev_number_filters, out_channels=number_filters, kernel_size=size_of_kernel)
        self.size = (self.size - size_of_kernel)
        prev_number_filters = number_filters
        number_filters = int(number_filters * filter_multiplier)
        self.activation_function_3 = activation_map.get(activation_function, torch.nn.ReLU())
        self.maxpooling_layer_3 = torch.nn.MaxPool2d(kernel_size=size_of_kernel, stride=2)
        self.size = self.size//2 # 50 x 16 x 16

        #layer4
        self.convolutional_layer_4 = torch.nn.Conv2d(in_channels=prev_number_filters, out_channels=number_filters, kernel_size=size_of_kernel)
        self.size = (self.size - size_of_kernel) # 50 x 14 x 14
        prev_number_filters = number_filters
        number_filters = int(number_filters * filter_multiplier)
        self.activation_function_4 = activation_map.get(activation_function, torch.nn.ReLU())
        self.maxpooling_layer_4 = torch.nn.MaxPool2d(kernel_size=size_of_kernel, stride=2)
        self.size = self.size//2

        #layer 5
        self.convolutional_layer_5 = torch.nn.Conv2d(in_channels=prev_number_filters, out_channels=number_filters, kernel_size=size_of_kernel)
        self.size = (self.size - size_of_kernel)
        prev_number_filters = number_filters
        stride_size=2
        self.activation_function_5 = activation_map.get(activation_function, torch.nn.ReLU())
        self.maxpooling_layer_5 = torch.nn.MaxPool2d(kernel_size=size_of_kernel, stride=stride_size)
        self.size = self.size//stride_size


        #Need to calculate the in_features.
        self.size = self.size * self.size * prev_number_filters
        size__=self.size
        self.fully_connected_layer_1 = torch.nn.Linear(in_features=size__, out_features=size__)
        self.dropout_layer_first = torch.nn.Dropout(dropout_factor)
        self.fully_connected_layer_2 = torch.nn.Linear(in_features=size__, out_features=number_of_classes)
        dimension_=1
        self.dropout_layer_second = torch.nn.Dropout(dropout_factor)
        self._softmax = torch.nn.LogSoftmax(dim=dimension_)

    def forward(self,data_x):
      return fn(self.convolutional_layer_1 ,self.activation_function_1 , self.maxpooling_layer_1 , self.convolutional_layer_2 ,self.activation_function_2 , self.maxpooling_layer_2 , self.convolutional_layer_3 , self.activation_function_3 ,  self.maxpooling_layer_3 , self.convolutional_layer_4 , self.activation_function_4 , self.maxpooling_layer_4 , self.convolutional_layer_5 ,self.activation_function_5 , self.maxpooling_layer_5 , self.fully_connected_layer_1,self.dropout_layer_first ,  self.fully_connected_layer_2,self.dropout_layer_second ,  self._softmax , data_x)


# Training the model

In [ ]:
def train(convolutional_neural_network, learning_rate, epochs, train_batch_load, val_batch_load,apply_wandb_log):
    loss_function = torch.nn.CrossEntropyLoss()
    epoch = 0
    optimizer = torch.optim.Adam(params=convolutional_neural_network.parameters(), lr=learning_rate, weight_decay = 1e-4)
    while epoch < epochs:

        acc_train = 0
        loss_train = 0
        bool__=1
        convolutional_neural_network.train()
        for i, (images, labels) in enumerate(train_batch_load):

            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  # doing zero gradient.
            predicted_y = convolutional_neural_network(images) #forward propogation
            loss = loss_function(predicted_y, labels) # Calculating Loss
            loss.backward()  # Backward Propagation
            optimizer.step() # update rule

            loss_train =loss_train + loss.item()

            _, prediction = torch.max(predicted_y.data, bool__)
            acc_train =acc_train + int(torch.sum(prediction == labels.data))

        acc_train =acc_train / len(glob.glob(train_data_directory+ '/**/*.jpg' ))
        loss_train = loss_train / len(glob.glob(train_data_directory+ '/**/*.jpg' ))
        print(f"Epochs : {epoch+1} Train Accuracy : {acc_train} Train Loss {loss_train}")

        with torch.no_grad():
            test_accuracy = 0
            test_loss = 0
            convolutional_neural_network.eval()
            for i, (images, labels) in enumerate(val_batch_load):
                images= images.to(device)

                labels =labels.to(device)
                predicted_y = convolutional_neural_network(images)

                loss = loss_function(predicted_y, labels)
                test_loss =test_loss+ loss.item()
                _, predicted = torch.max(predicted_y.data, 1)
                test_accuracy = test_accuracy + int(torch.sum(predicted == labels.data))

            test_accuracy = test_accuracy/ len(glob.glob(test_data_directory+ '/**/*.jpg' ))
            test_loss = test_loss /  len(glob.glob(test_data_directory+ '/**/*.jpg' ))

            print(f"Epochs : {epoch+1} Validation Accuracy : {test_accuracy} Validation Loss {test_loss}")
            if(apply_wandb_log):
                wandb.log({"train_accuracy": acc_train, "train_loss" : loss_train, "val_accuracy": test_accuracy, "val_error": test_loss})

        epoch += 1

## Function to get the Test Accuracy & Test Loss

In [ ]:
def test_print(convolutional_neural_network , test_data_directory,epochs,test_batch_load , loss_function , count_test ):
    for epoch in range(epochs):

        with torch.no_grad():
            convolutional_neural_network.eval()
            test_loss = 0
            test_accuracy = 0
            for i, (images, labels) in enumerate(test_batch_load):

                images, labels = images.to(device), labels.to(device)

                loss = loss_function(convolutional_neural_network(images), labels)
                test_loss = test_loss +  loss.item()


                _, predicted = torch.max(convolutional_neural_network(images).data,1)
                test_accuracy = test_accuracy + int(torch.sum(predicted == labels.data))
            test_accuracy = test_accuracy / count_test
            test_loss = test_loss / count_test

            print(f"Epochs : {epoch+1} Test Accuracy : {test_accuracy} Test Loss {test_loss}")

In [ ]:
def CNN_test(convolutional_neural_network, number_of_classes, test_data_directory, test_batch_size , num_of_filters, kernel_size, activation_function, filter_multiplier, learning_rate, epochs, dropout_factor , image_size):


    transformer = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor()
    ])

    test_Dataset = torchvision.datasets.ImageFolder(root=test_data_directory, transform=transformer)
    test_batch_load = DataLoader(
    test_Dataset,
    batch_size=test_batch_size,
    shuffle=True)
    count_test = len(glob.glob(test_data_directory+'/**/*.jpg'))
    print('star=>')
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=convolutional_neural_network.parameters(), lr=learning_rate)
    count_test = len(glob.glob(test_data_directory+'/**/*.jpg'))
    test_print(convolutional_neural_network , test_data_directory,epochs,test_batch_load , loss_function, count_test)

In [ ]:
def main(apply_data_augmentation ,number_of_classes, kernel_size,  train_data_directory, test_data_directory, train_batch_size, val_batch_size, test_batch_size, num_of_filters, activation_function, filter_multiplier,learning_rate, epochs, apply_wandb_log, dropout_factor , image_size):
    mean=[0.4602, 0.4495, 0.3800]
    print("start=>")
    std=[0.2040, 0.1984, 0.1921]
    train_batch_load, val_batch_load, test_batch_load = data_fetch(apply_data_augmentation, train_data_directory, test_data_directory, train_batch_size, val_batch_size, test_batch_size , image_size , mean , std)
    convolutional_neural_network = initialize_lenet_CNN(number_of_classes, kernel_size, num_of_filters, activation_function, filter_multiplier, dropout_factor)
    convolutional_neural_network = convolutional_neural_network.to(device)
    train(convolutional_neural_network, learning_rate, epochs, train_batch_load, val_batch_load,apply_wandb_log)
    print("Done!!")
    return convolutional_neural_network

## Running Sweep

In [ ]:
def sweep_():
    train_batch_size = 64
    test_batch_size = 16
    val_batch_size = 16
    number_of_classes = 10
    kernel_size = 3
    apply_wandb_log = True
    apply_data_augmentation=True

    parameters_by_default ={
        'num_of_filters' : 32,
        'activation_function' : 'ReLU',
        'filter_multiplier' : 0.2,
        'learning_rate' : 0.001,
        'epochs' : 3,
        'dropout_factor' : 0,
        "image_size" : 256,
        "apply_data_augmentation": True
    }

    run = wandb.init(config=parameters_by_default, project='Deep_Learning_A2')

    config = wandb.config
    img_sz=(config.image_size,config.image_size)
    run.name = 'ac_' + config.activation_function + '_ep_' + str(config.epochs) +  '_ff_'+ str(config.filter_multiplier) + '_nf_' + str(config.num_of_filters) +'_df_'+str(config.dropout_factor)
    main(config.apply_data_augmentation ,number_of_classes, kernel_size,  train_data_directory, test_data_directory, train_batch_size, val_batch_size, test_batch_size, config.num_of_filters, config.activation_function, 0.5,config.learning_rate, config.epochs, apply_wandb_log, config.dropout_factor , (256,256))

#Set up a sweep config

In [ ]:
config = {
    'name': 'cs23m013',
    'metric': {
        'goal': 'maximize',
        'name': 'val_accuracy'
        },
    "method": "random",
    "project": "Deep_Learning_A2",
    "parameters": {

         "image_size" : {
            "values": [224, 256]
        },
         "epochs": {
            "values": [5,10]
        },
        "activation_function":
        {
            "values" : ["LeakyReLU" ,"Mish", "SiLU", "GELU", "ReLU"]
        },
        "dropout_factor":
        {
            "values": [0 ,0.1 , 0.2 , 0.3, 0.4]
        },
        "num_of_filters": {
            "values": [32, 64]
        },
        "filter_multiplier": {
            "values": [0.3 , 0.4 , 0.5]
        },
        "learning_rate" : {
            "values" : [0.0001, 0.0003]
            },
        "apply_data_augmentation" : {
            "values" : [True]
            },

    }
}


# creating the sweep


In [ ]:
sweep_id = wandb.sweep(config, project="Deep_Learning_A2")
wandb.agent(sweep_id, function=sweep_, count=1)

Create sweep with ID: zktrkxoz
Sweep URL: https://wandb.ai/cs23m013/Deep_Learning_A2/sweeps/zktrkxoz


wandb: Agent Starting Run: iwxoysmw with config:
wandb: 	activation_function: LeakyReLU
wandb: 	apply_data_augmentation: True
wandb: 	dropout_factor: 0.2
wandb: 	epochs: 5
wandb: 	filter_multiplier: 0.3
wandb: 	image_size: 256
wandb: 	learning_rate: 0.0003
wandb: 	num_of_filters: 64
wandb: Currently logged in as: cs23m013. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


start=>
Epochs : 1 Train Accuracy : 0.0 Train Loss 0.11584246158599854
Epochs : 1 Validation Accuracy : 0.05 Validation Loss 0.11464235782623292
Epochs : 2 Train Accuracy : 0.1 Train Loss 0.11539503335952758
Epochs : 2 Validation Accuracy : 0.05 Validation Loss 0.11478745937347412
Epochs : 3 Train Accuracy : 0.0 Train Loss 0.1155168056488037
Epochs : 3 Validation Accuracy : 0.05 Validation Loss 0.11493200063705444
Epochs : 4 Train Accuracy : 0.1 Train Loss 0.11499874591827393
Epochs : 4 Validation Accuracy : 0.05 Validation Loss 0.11512091159820556
Epochs : 5 Train Accuracy : 0.05 Train Loss 0.11498440504074096
Epochs : 5 Validation Accuracy : 0.05 Validation Loss 0.11526086330413818
Done!!


train_accuracy,▁█▁█▅
train_loss,█▄▅▁▁
val_accuracy,▁▁▁▁▁
val_error,▁▃▄▆█
train_accuracy,0.05
train_loss,0.11498
val_accuracy,0.05
val_error,0.11526


#Testing on the best hyperparameters

In [ ]:
def best_model(apply_data_augmentation ,test_batch_size ,train_batch_size , val_batch_size , number_of_classes  , kernel_size  , num_of_filters  , filter_multiplier  , learning_rate   ,epochs  ,dropout_factor  ,  activation_function  , apply_wandb_log , image_size):
    convolutional_neural_network = main(apply_data_augmentation ,number_of_classes, kernel_size,  train_data_directory, test_data_directory, train_batch_size, val_batch_size, test_batch_size, num_of_filters, activation_function, filter_multiplier, learning_rate, epochs, apply_wandb_log, dropout_factor , image_size)
    CNN_test(convolutional_neural_network, number_of_classes, test_data_directory, test_batch_size , num_of_filters, kernel_size, activation_function, filter_multiplier, learning_rate, epochs, dropout_factor ,image_size)

In [ ]:
best_model(apply_data_augmentation=True , test_batch_size = 16 ,train_batch_size = 64 , val_batch_size = 16 , number_of_classes = 10 , kernel_size = 3 , num_of_filters = 32 , filter_multiplier = 1 , learning_rate = 0.001  ,epochs = 10 ,dropout_factor = 0.3 ,  activation_function = 'GELU' , apply_wandb_log = False , image_size=(256,256))

#Plotting 3 X 10 images

In [ ]:
# classes=('Amphibia','Animalia','Arachnida','Aves','Fungi','Insecta','Mammalia','Mollusca','Plantae','Reptilia')

In [ ]:
# def my_plot_model(apply_data_augmentation ,test_batch_size ,train_batch_size , val_batch_size , number_of_classes  , kernel_size  , num_of_filters  , filter_multiplier  , learning_rate   ,epochs  ,dropout_factor  ,  activation_function  , apply_wandb_log , image_size):
#     return main(apply_data_augmentation ,number_of_classes, kernel_size,  train_data_directory, test_data_directory, train_batch_size, val_batch_size, test_batch_size, num_of_filters, activation_function, filter_multiplier, learning_rate, epochs, apply_wandb_log, dropout_factor , image_size)

In [ ]:
# model_plot=my_plot_model(apply_data_augmentation=True , test_batch_size = 16 ,train_batch_size = 64 , val_batch_size = 16 , number_of_classes = 10 , kernel_size = 3 , num_of_filters = 32 , filter_multiplier = 1 , learning_rate = 0.001  ,epochs = 1 ,dropout_factor = 0.3 ,  activation_function = 'GELU' , apply_wandb_log = False , image_size=(256,256))

In [ ]:
# def trans_form_image_dataset(image_size=(256, 256)):
#           transformer = transforms.Compose([
#                       transforms.Resize(image_size),
#               transforms.ToTensor(),
#                   ])
#           test_Dataset = torchvision.datasets.ImageFolder(root=test_data_directory, transform=transformer)
#           return test_Dataset

In [ ]:
# model=model_plot
# loader_val = DataLoader(trans_form_image_dataset((256, 256)),16,True)
# model.eval()
# pridicted_label = []
# actual_label = []
# with torch.no_grad():
#     for images, _ in loader_val:
#         images = images.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs, 1)
#         pridicted_label.extend(predicted.cpu().numpy())


# class_of_images = [[] for _ in range(len(loader_val.dataset.classes))]
# image_count = [0] * len(class_of_images)

# with torch.no_grad():
#     for img, label in loader_val:
#         for image, label in zip(img, label):
#             if image_count[label] <= 2:
#                 class_of_images[label].append(image)
#                 actual_label.append(label)
#                 image_count[label] = image_count[label]+  1

# label_names=classes
# fig, axs = plt.subplots(10, 3, figsize=(15, 30))
# fig.tight_layout(pad=3.0)

# for idx_i in range(len(class_of_images)):
#     idx_j = 0
#     while idx_j < 3:
#         img = class_of_images[idx_i][idx_j].permute(1, 2, 0)
#         predicted_label = label_names[predicted_label[idx_i * 3 + idx_j]]
#         actual_label = label_names[actual_label[idx_i]]
#         axs[idx_i, idx_j].imshow(img)
#         axs[idx_i, idx_j].set_title(f"Actual: {actual_label} \n Predicted: {predicted_label}", fontsize=15, color='black', backgroundcolor='lightgrey', pad=11)
#         axs[idx_i, idx_j].set_xticks([])
#         axs[idx_i, idx_j].set_yticks([])
#         axs[idx_i, idx_j].spines['top'].set_color('none')
#         axs[idx_i, idx_j].spines['bottom'].set_color('none')
#         axs[idx_i, idx_j].spines['left'].set_color('none')
#         axs[idx_i, idx_j].spines['right'].set_color('none')
#         axs[idx_i, idx_j].tick_params(axis='both', which='both', length=0)
#         axs[idx_i, idx_j].set_aspect('equal')
#         axs[idx_i, idx_j].grid(color='grey', linestyle='--', linewidth=0.5)
#         idx_j = idx_j + 1

# plt.show()